# **Advanced Data Analysis Queries**

In [ ]:
-- 1. Change-over-Time: Monthly Fleet Utilization Trend

SELECT
    date_format(acquisition_date, 'yyyy-MM') AS month,
    ROUND(AVG(rental_count * 1.0), 2) AS avg_rentals_per_vehicle
FROM vehicle_lifecycle
GROUP BY date_format(acquisition_date, 'yyyy-MM')
ORDER BY month;

In [ ]:
-- 2. Cumulative Analysis: YTD Revenue by Branch

SELECT
    c.preferred_branch AS branch,
    year(cu.registration_date) AS year,
    SUM(c.total_rental_revenue + c.total_lease_value) AS ytd_revenue
FROM customer_360 c
JOIN ZedaFleet_silver.customers cu
    ON c.customer_id = cu.customer_id
GROUP BY c.preferred_branch, year(cu.registration_date)
ORDER BY year, ytd_revenue DESC;

In [ ]:
-- 3. Performance Analysis: Maintenance Cost vs Operating Profit

SELECT
    v.vehicle_id,
    v.make,
    v.model,
    v.rental_revenue + v.lease_revenue + COALESCE(v.sale_price,0) AS total_operating_revenue,
    ROUND(v.total_maintenance_cost, 2) AS total_maintenance,
    (v.rental_revenue + v.lease_revenue + COALESCE(v.sale_price,0)) - v.total_maintenance_cost AS operating_profit
FROM vehicle_lifecycle v
ORDER BY operating_profit DESC;

In [ ]:
-- 4. Part-to-Whole: Revenue Contribution by Branch (Annual)

SELECT
    c.preferred_branch AS branch,
    year(cu.registration_date) AS year,
    SUM(c.total_rental_revenue + c.total_lease_value) AS branch_revenue,
    ROUND(
        SUM(c.total_rental_revenue + c.total_lease_value) * 100.0 /
        SUM(SUM(c.total_rental_revenue + c.total_lease_value)) OVER (PARTITION BY year(cu.registration_date)), 2
    ) AS pct_contribution
FROM customer_360 c
JOIN ZedaFleet_silver.customers cu
    ON c.customer_id = cu.customer_id
GROUP BY c.preferred_branch, year(cu.registration_date)
ORDER BY year, pct_contribution DESC;

In [ ]:
-- 5. Data Segmentation: Utilization Rate by Vehicle Segment

SELECT
    v.segment,
    COUNT(v.vehicle_id) AS fleet_size,
    SUM(v.rental_count) AS total_rentals,
    ROUND(AVG(v.rental_count * 1.0), 2) AS avg_utilization_rate
FROM vehicle_lifecycle v
GROUP BY v.segment
ORDER BY avg_utilization_rate DESC;